# Coeficiente de Lyapunov
En los anteriores archivos hemos visto que efectivamente el **Atractor de Lorenz** es sensible a pequeñas perturbaciones sobre las condicionales (de la misma forma para los parámetros $\sigma$, $\rho$ y $\beta$). ¿Habrá alguna forma de medir esa sensibilidad? ¿Habrá alguna manera de, sin resolver el sistema o sin hacer perturbaciones, podemos decretar que es un sistema complejo? O al menos, ¿habrá alguna forma de decretarlo?

Este concepto nace de la mente Aleksandr Lyapunov a finales del siglo XIX, en el contexto de la estabilidad de sistemas dinámicos. Su trabajo sentó las bases para el estudio moderno del caos y la teoría de sistemas complejos. Más tarde, con el desarrollo de la computación y el análisis numérico, los exponentes de Lyapunov se convirtieron en una herramienta fundamental para caracterizar el comportamiento caótico en sistemas físicos, meteorológicos y matemáticos, como el famoso *atractor de Lorenz* descubierto en 1963. 

**Pero ¿Qué es el coeficiente de Lyapunov?** El coeficiente de Lyapunov mide la sensibilidad a las condiciones iniciales en sistemas dinámicos, como el atractor de Lorenz. Específicamente, cuantifica la tasa de divergencia exponencial entre trayectorias cercanas: si el coeficiente es positivo, pequeñas diferencias iniciales se amplifican rápidamente, lo que indica comportamiento caótico. Para el atractor de Lorenz, el mayor exponente de Lyapunov es positivo, lo que confirma la naturaleza caótica del sistema y la imposibilidad de predecir su evolución a largo plazo.

La fórmula para calcular el coeficiente de Lyapunov es:
$$
\lambda = \lim_{t \to \infty} \frac{1}{t} \ln \left( \frac{|\delta(t)|}{|\delta_0|} \right)
$$

donde $\delta_0$ es la distancia inicial entre dos trayectorias y $\delta(t)$ es la distancia en el tiempo $t$. Si $\lambda > 0$, el sistema es caótico.

Como $\lim{t \to \infty}$ y eso está *muy lejos*, vamos a ver como evoluciona $\lambda$ para diferentes tiempos para ver si tiende a algun valor en específico.

De manera práctica, vamos a seguir los siguientes pasos: 
- Calculamos las trayectorias del sistema sin perturbador x1 y el sistema perturbado x2 a partir del **Método de RK4** y $\delta _0$
- Calculamos la separación $\delta$ como el modulo de la diferencia de las trayectorias en el mismo punto del tiempo y lo que haremos será acumular la en S el logaritmo de esta: 
 $$
S += ln(\frac{\delta}{\delta _0})
 $$
- Por último, renormalizamos $\delta _0$ manteniendo la dirección de $\delta$

Este proceso lo haremos hasta llegar a nuestro t_final = T, un vector que va de 200 a 4000 en 100 pasos.

$$
    \lambda = \frac{1}{N} \sum ^{N}_{i=1} ln (\frac{\delta _k}{\delta _0}) = \frac{S}{T}
$$

In [20]:
import numpy as np
import math as m
import matplotlib.pyplot as plt

Definimos las funciones que vamos a necesitar para el programa: el sistema de Lorentz, el método de Runge Kutta y coef_lyapunov

In [ ]:
def rk4_step(f, x, h): #Función para un paso de Runge-Kutta de cuarto orden
    k1 = f(x)
    k2 = f(x + 0.5*h*k1)
    k3 = f(x + 0.5*h*k2)
    k4 = f(x + h*k3)
    return x + (h/6.0)*(k1 + 2*k2 + 2*k3 + k4)

def lorenz(x, sigma=10., beta=8/3, rho=28.): # Sistema de Lorenz
    dx = sigma*(x[1] - x[0])
    dy = x[0]*(rho - x[2]) - x[1]
    dz = x[0]*x[1] - beta*x[2]
    return np.array([dx, dy, dz])

def lyapunov_lorenz(x0, delta0=1e-8, h=0.01, t_final=100.0, renorm_interval=0.1): #Funcion para calcular el exponente de Lyapunov
    
    x1 = np.array(x0, dtype=float) #Trayectoria de la partícula
    
    v = np.random.normal(size=3) # Vector aleatorio para la perturbación inicial
    v /= np.linalg.norm(v) # Normalizar el vector
    x2 = x1 + delta0 * v #Trayectoria de la partícula perturbada

    S = 0.0  # Acmulará el factor de dilatación de ambas trayectorias
    t = 0.0
    steps_per_renorm = int(renorm_interval / h) # Número de pasos entre renormalizaciones
    total_steps = int(t_final / h) # Número total de pasos

    for i in range(1, total_steps + 1):
        x1 = rk4_step(lorenz, x1, h)
        x2 = rk4_step(lorenz, x2, h)
        if i % steps_per_renorm == 0: #El % en el condicional es el operador módulo, que devuelve el resto de la división
            diff = x2 - x1
            dist = np.linalg.norm(diff)
            S += np.log(dist / delta0)
            diff = (delta0 / dist) * diff
            x2 = x1 + diff
            t += renorm_interval
    return S / (t)

In [22]:
x0 = [0.0, 1.0, 0.0] #Condicion inicial

t_final = np.linspace(200, 4000, 100) #Tiempos de simulación
lambda_max = []

for T in t_final:
    λ = lyapunov_lorenz(x0, delta0=1e-8, h=0.5, t_final=T, renorm_interval=0.05)
    lambda_max.append(λ)

KeyboardInterrupt: 

In [23]:
print("Exponente de Lyapunov λ:", lambda_max[len(lambda_max)])
plt.figure(figsize=(10, 6))
plt.plot(t_final, lambda_max, marker='*', label='λ vs T')
plt.xlabel('Tiempo de integración T')
plt.ylabel('Exponente de Lyapunov λ')
plt.grid(True)
plt.legend()
plt.show()

IndexError: list index out of range